# Pengumpulan Data Repository GitHub

Notebook ini berfungsi sebagai tahap awal (mining) untuk mengambil data mentah dari GitHub. Saya akan menggunakan GitHub API untuk mencari repository yang relevan, mengambil detailnya, dan menyimpannya ke dalam bentuk dataset (CSV) agar bisa saya analisis lebih lanjut.

In [1]:
# cek library di venv

import importlib

libraries = [
    'requests', 
    'pandas', 
    'dotenv', 
    'sklearn', 
    'matplotlib', 
    'seaborn', 
    'notebook', 
    'ipykernel', 
    'tqdm'
]

print("memulai pengecekan library...")
print("-" * 30)

all_safe = True

for lib in libraries:
    try:
        importlib.import_module(lib)
        print(f"{lib} sudah terinstall")
    except ImportError:
        print(f"{lib} belum terinstall atau tidak terbaca")
        all_safe = False

print("-" * 30)
if all_safe:
    print("semua library sudah terinstall.")
else:
    print("ada library yang belum terinstall")

memulai pengecekan library...
------------------------------
requests sudah terinstall
pandas sudah terinstall
dotenv sudah terinstall
sklearn sudah terinstall
matplotlib sudah terinstall
seaborn sudah terinstall
notebook sudah terinstall
ipykernel sudah terinstall
tqdm sudah terinstall
------------------------------
semua library sudah terinstall.


In [2]:
# Setup Environment dan Load Token

# Tahap pertama adalah mengimpor library sistem yang dibutuhkan dan mengambil kredensial akses. 
# Saya menggunakan library dotenv untuk memuat environment variable dari file .env ke dalam sesi Python saat ini.

import os
from dotenv import load_dotenv

load_dotenv()

token_github = os.getenv("GITHUB_TOKEN")

if token_github:
    print("Token berhasil dimuat.")
    print(f"Prefix token: {token_github[:4]}...")
else:
    print("Token tidak ditemukan.")

Token berhasil dimuat.
Prefix token: ghp_...


### Validasi Token

Output "token berhasil dimuat" mengonfirmasi bahwa skrip berhasil membaca GITHUB_TOKEN dari environment lokal. Variabel token_github kini menyimpan string token yang valid. Dengan adanya token ini, setiap request yang dikirimkan ke GitHub API nantinya akan memiliki limit akses (rate limit) yang lebih tinggi dibandingkan akses anonim, serta mengizinkan akses ke data repository yang mungkin memerlukan izin khusus.

In [3]:
# Import Library Data dan Konfigurasi Header HTTP

# Setelah token siap, saya mengimpor library utama yang akan digunakan untuk proses mining. 
# Saya menggunakan requests untuk mengirim permintaan HTTP ke server GitHub dan `pandas` untuk 
# memproses data yang diterima menjadi bentuk tabel (DataFrame). Selain itu, 
# saya juga menyusun `header` permintaan yang berisi token otentikasi. Ini diperlukan agar GitHub 
# mengenali identitas pengakses dan memberikan izin akses data yang lebih luas.

import requests
import pandas as pd
import time

header_api = {
    "Authorization": f"token {token_github}",
    "Accept": "application/vnd.github.v3+json"
}

print("Konfigurasi header dan library selesai.")

Konfigurasi header dan library selesai.


### Kesiapan Alat

Variabel header_api kini telah berisi dictionary yang memuat protokol otentikasi. Dengan header ini, setiap permintaan yang dikirimkan menggunakan library requests akan dianggap sebagai permintaan resmi dari akun pemilik token, bukan pengguna anonim. Saya juga menetapkan parameter Accept untuk memastikan format respons yang diterima sesuai dengan standar API GitHub versi 3 (v3). Library pandas juga siap digunakan untuk menampung data JSON yang nanti didapatkan.

In [4]:
# Membuat Fungsi Pencarian Repository

# Saya akan membuat sebuah fungsi python yang bertugas secara spesifik untuk mengirim permintaan pencarian ke GitHub. 
# Fungsi ini dirancang agar dinamis, artinya saya bisa mengganti kata kunci pencarian atau nomor halaman kapan saja 
# tanpa harus mengubah struktur kode utamanya. 
# Fungsi ini juga mengatur parameter agar hasil pencarian diurutkan berdasarkan popularitas atau jumlah bintang.

def cari_repository(kata_kunci, halaman):
    url_dasar = "https://api.github.com/search/repositories"
    parameter = {
        "q": kata_kunci,
        "sort": "stars",
        "order": "desc",
        "per_page": 10,
        "page": halaman
    }

    respon = requests.get(url_dasar, headers=header_api, params=parameter)

    if respon.status_code == 200:
        return respon.json()
    else:
        print(f"Gagal. Kode status {respon.status_code}")
        return None

print("Fungsi pencarian repository siap digunakan.")

Fungsi pencarian repository siap digunakan.


### Logika Fungsi Pencarian

Fungsi cari_repository di atas sekarang sudah tersimpan di memori. Fungsi ini menerima dua input, yaitu kata kunci topik dan nomor halaman. Saya telah mengatur parameter sort menjadi stars dan order menjadi desc agar repository yang diambil adalah yang paling populer (bintang terbanyak) di GitHub. Jika komunikasi dengan server berhasil (kode 200), fungsi akan mengembalikan data mentah, namun jika gagal, fungsi akan memberi laporan kode error.

In [5]:
# Eksekusi Pencarian Data

# Sekarang saya akan menggunakan fungsi yang telah dibuat untuk mengambil data nyata. 
# Saya menetapkan topik pencarian machine learning dan akan mengambil data dari 3 halaman pertama sebagai sampel awal. 
# Setiap kali selesai mengambil satu halaman, 
# saya menambahkan jeda waktu satu detik agar server GitHub tidak memblokir akses karena terlalu banyak permintaan dalam waktu singkat.

topik = "Machine Learning"
total_halaman = 3
data_mentah = []

print(f"Memulai pencarian untuk topik: {topik}")

for i in range(1, total_halaman + 1):
    hasil = cari_repository(topik, i)
    if hasil:
        items = hasil.get("items", [])
        data_mentah.extend(items)
        print(f"Halaman {i} berhasil diambil. Total data saat ini: {len(data_mentah)}")
    time.sleep(1)

print("Proses pengambilan data selesai.")

Memulai pencarian untuk topik: Machine Learning
Halaman 1 berhasil diambil. Total data saat ini: 10
Halaman 2 berhasil diambil. Total data saat ini: 20
Halaman 3 berhasil diambil. Total data saat ini: 30
Proses pengambilan data selesai.


### Hasil Pengumpulan Data Mentah

Proses perulangan di atas telah berhasil mengumpulkan sekumpulan data JSON dari GitHub. Variabel data_mentah sekarang berisi daftar repository yang relevan dengan topik machine learning yang diurutkan berdasarkan jumlah bintang. Jeda waktu yang diterapkan membantu menjaga koneksi tetap stabil dan menghindari penolakan dari server. Data ini masih dalam bentuk daftar mentah dan perlu dirapikan ke dalam tabel agar lebih mudah dianalisis.

In [7]:
# Pembersihan dan Penyimpanan Data
# Setelah data mentah terkumpul, saya perlu memilah informasi yang penting saja. 
# Saya akan membuat list baru yang berisi dictionary dengan key yang lebih rapi, lalu mengubahnya menjadi dataframe pandas. 
# Terakhir, saya akan menyimpan hasilnya ke file csv agar tidak hilang

data_bersih = []

for butir in data_mentah:
    data_bersih.append({
        "Nama repository": butir['name'],
        "Nama lengkap": butir['full_name'],
        "Pemilik": butir['owner']['login'],
        "Jumlah bintang": butir['stargazers_count'],
        "Jumlah fork": butir['forks_count'],
        "Jumlah isu terbuka": butir['open_issues_count'],
        "Bahasa pemrograman": butir['language'],
        "Tanggal dibuat": butir['created_at'],
        "Tanggal diperbarui": butir['updated_at'],
        "Tautan url": butir['html_url'],
        "Deskripsi": butir['description']
    })

tabel_repository = pd.DataFrame(data_bersih)

print("Lima data teratas")
display(tabel_repository.head())

print("Informasi tipe data")
print(tabel_repository.info())

nama_berkas = "../data/raw_data_machine_learning.csv"
tabel_repository.to_csv(nama_berkas, index=False)

print(f"Sukses menyimpan data ke {nama_berkas}")

Lima data teratas


,Nama repository,Nama lengkap,Pemilik,Jumlah bintang,Jumlah fork,Jumlah isu terbuka,Bahasa pemrograman,Tanggal dibuat,Tanggal diperbarui,Tautan url,Deskripsi
0,tensorflow,tensorflow/tensorflow,tensorflow,193087,75157,2588,C++,2015-11-07T01:19:20Z,2025-12-28T19:23:50Z,https://github.com/tensorflow/tensorflow,An Open Source Machine Learning Framework for ...
1,transformers,huggingface/transformers,huggingface,154348,31563,2163,Python,2018-10-29T13:56:00Z,2025-12-28T19:27:54Z,https://github.com/huggingface/transformers,🤗 Transformers: the model-definition framework...
2,ML-For-Beginners,microsoft/ML-For-Beginners,microsoft,82651,19460,2,Jupyter Notebook,2021-03-03T01:34:05Z,2025-12-28T19:19:26Z,https://github.com/microsoft/ML-For-Beginners,"12 weeks, 26 lessons, 52 quizzes, classic Mach..."
3,funNLP,fighting41love/funNLP,fighting41love,78123,15109,43,Python,2018-08-21T11:20:39Z,2025-12-28T15:15:42Z,https://github.com/fighting41love/funNLP,中英文敏感词、语言检测、中外手机/电话归属地/运营商查询、名字推断性别、手机号抽取、身份证抽...
4,awesome-machine-learning,josephmisiti/awesome-machine-learning,josephmisiti,71110,15217,23,Python,2014-07-15T19:11:19Z,2025-12-28T19:05:31Z,https://github.com/josephmisiti/awesome-machin...,A curated list of awesome Machine Learning fra...


Informasi tipe data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Nama repository     30 non-null     object
 1   Nama lengkap        30 non-null     object
 2   Pemilik             30 non-null     object
 3   Jumlah bintang      30 non-null     int64 
 4   Jumlah fork         30 non-null     int64 
 5   Jumlah isu terbuka  30 non-null     int64 
 6   Bahasa pemrograman  22 non-null     object
 7   Tanggal dibuat      30 non-null     object
 8   Tanggal diperbarui  30 non-null     object
 9   Tautan url          30 non-null     object
 10  Deskripsi           30 non-null     object
dtypes: int64(3), object(8)
memory usage: 2.7+ KB
None
Sukses menyimpan data ke ../data/raw_data_machine_learning.csv


### Status dataset

Data json yang tadinya acak sekarang sudah tersusun rapi dalam bentuk tabel. Saya sudah memverifikasi baris teratas dan tipe datanya untuk memastikan tidak ada error. File csv hasil olahan ini sekarang tersimpan di folder data dan siap saya gunakan untuk analisis visualisasi di tahap berikutnya.